In [2]:
import glob
import os
from shutil import copyfile

import scipy.io as sio

import cv2
import numpy as np
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm

from misc.patch_extractor import PatchExtractor
from misc.utils import rm_n_mkdir

from config import Config

In [3]:
def bounding_box(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out 
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]

def draw_contours(mask, ann_inst, line_thickness=1):
    overlay = np.copy((mask).astype(np.uint8))

    label_map = ann_inst
    instances_list = list(np.unique(label_map))  # get list of instances
    instances_list.remove(0)  # remove background
    contours = []
    for inst_id in instances_list:
        instance_map = np.array(
            ann_inst == inst_id, np.uint8)  # get single object
        y1, y2, x1, x2 = bounding_box(instance_map)
        y1 = y1 - 2 if y1 - 2 >= 0 else y1
        x1 = x1 - 2 if x1 - 2 >= 0 else x1
        x2 = x2 + 2 if x2 + 2 <= ann_inst.shape[1] - 1 else x2
        y2 = y2 + 2 if y2 + 2 <= ann_inst.shape[0] - 1 else y2
        inst_map_crop = instance_map[y1:y2, x1:x2]
        contours_crop = cv2.findContours(
            inst_map_crop, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        index_correction = np.asarray([[[[x1, y1]]]])
        for i in range(len(contours_crop[0])):
            contours.append(
                list(np.asarray(contours_crop[0][i].astype('int32')) + index_correction))
    contours = list(itertools.chain(*contours))
    cv2.drawContours(overlay, np.asarray(contours), -1, 2, line_thickness)
    return overlay

In [3]:
img_ext = '.png'
label_ext = '.mat'

In [5]:
cfg = Config()

extract_type = 'mirror' # 'valid' for fcn8 segnet etc.
                        # 'mirror' for u-net etc.
# check the patch_extractor.py 'main' to see the different

# orignal size (win size) - input size - output size (step size)
# 512x512 - 256x256 - 256x256 fcn8, dcan, segnet
# 536x536 - 268x268 - 84x84   unet, dist
# 540x540 - 270x270 - 80x80   xy, hover
# 504x504 - 252x252 - 252x252 micronetcd tr
step_size = [256, 256] # should match self.train_mask_shape (config.py) 
win_size  = [512, 512] # should be at least twice time larger than 
                       # self.train_base_shape (config.py) to reduce 
                       # the padding effect during augmentation

xtractor = PatchExtractor(win_size, step_size)

### Paths to data - these need to be modified according to where the original data is stored
img_ext = '.png'
img_dir = '/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Images/'
ann_dir = '/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Labels/' 
####
out_dir = "/home1/gzy/NucleiSegmentation/High_CCRCC/Test/%dx%d_%dx%d" % \
                    (win_size[0], win_size[1], step_size[0], step_size[1])

file_list = glob.glob('%s/*%s' % (img_dir, img_ext))
file_list.sort()

rm_n_mkdir(out_dir)

['/home1/gzy/NucleiSegmentation/High_CCRCC/Train/512x512_256x256_dist/']
['/home1/gzy/NucleiSegmentation/High_CCRCC/Valid/512x512_256x256_dist/']


In [6]:
for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = filename.split('.')[0]
    #print(filename)

    img = cv2.imread(img_dir + basename + img_ext)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    labels = sio.loadmat(ann_dir + basename + '.mat')
    ann_inst = labels['instance_map']   
    ann_type = labels['class_map']
#     ann_pos = labels['positive_map'] 
#     ann_neg = labels['negative_map']
#     ann_dist = labels['dist_map']*255
#     ann_marker = labels['marker_map']*255
    
    ann = np.dstack([ann_inst, ann_type])
    ann = ann.astype('int32')
    
    img = np.concatenate([img, ann], axis=-1)
    sub_patches = xtractor.extract(img, extract_type)
    for idx, patch in enumerate(sub_patches):
        np.save("{0}/{1}_{2:03d}.npy".format(out_dir, basename, idx), patch)

100%|██████████| 200/200 [00:40<00:00,  4.97it/s]


In [6]:
# save_dir = "/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Overlays"
# for filename in tqdm(file_list):
#     filename = os.path.basename(filename)
#     basename = filename.split('.')[0]
#     #print(filename)

#     img = cv2.imread(img_dir + basename + img_ext)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     labels = sio.loadmat(ann_dir + basename + '.mat')
#     ann_inst = labels['instance_map']  
#     ann_inst[ann_inst!=0] = 255
#     ann_marker = labels['marker_map']*255
#     cv2.imwrite('%s/%s_dist.png' % (save_dir+'/dist_map/', basename), ann_marker)
#     cv2.imwrite('%s/%s_bd.png' % (save_dir+'/binary_map/', basename), ann_inst)

100%|██████████| 200/200 [00:04<00:00, 48.88it/s]
